In [ ]:
cd /content/drive/MyDrive/MBKM RISET/RISET_1

/content/drive/MyDrive/MBKM RISET/RISET_1


## Loading Images in Dataset

In [ ]:
import os
path_live = "/content/drive/MyDrive/MBKM RISET/RISET_1/SpecDiff_in_house_database_sample/data/live"
path_spoof = "/content/drive/MyDrive/MBKM RISET/RISET_1/SpecDiff_in_house_database_sample/data/spoof"

In [ ]:
def load_images(path):
  image_path_flash = []
  image_path_background = []
  for file_id in os.listdir(path):
    id = os.path.join(path,file_id)
    counter = True
    for filename in os.listdir(id):
      if counter == True:
        image_path_background.append(os.path.join(id, filename))
        counter = False
      elif counter == False: 
        image_path_flash.append(os.path.join(id, filename))
        counter = True
  return image_path_flash,image_path_background

In [ ]:
live_flash,live_bg = load_images(path_live)
spoof_flash,spoof_bg = load_images(path_spoof)
sum_live = len(live_flash)+len(live_bg)
sum_spoof = len(spoof_flash)+len(spoof_bg)
print("Jumlah Pasangan Gambar Live : " + str(sum_live/2))
print("Jumlah Pasangan Gambar Spoof : " + str(sum_spoof/2))

Jumlah Pasangan Gambar Live : 240.0
Jumlah Pasangan Gambar Spoof : 336.0


In [ ]:
print(live_flash[0:10])
print(live_bg[0:10])
print(spoof_flash[0:10])
print(spoof_bg[0:10])

## Library

In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat \
    -O shape_predictor_68_face_landmarks.dat
    
from imutils import face_utils
import imutils
import numpy as np
import collections
import dlib
import cv2

%matplotlib inline 
from matplotlib import pyplot as plt 
import pylab 
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

--2021-10-29 06:55:56--  https://raw.githubusercontent.com/italojs/facial-landmarks-recognition/master/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[===================>]  95.08M  60.8MB/s    in 1.6s    

2021-10-29 06:56:01 (60.8 MB/s) - ‘shape_predictor_68_face_landmarks.dat’ saved [99693937/99693937]



## Function

In [ ]:
base_path = "/content/drive/MyDrive/MBKM RISET/"

def cropping(image):
  try:
    rect = detector(image)[0]
  except (ValueError,IndexError):
    print("Not Found Face!!")
    return image
  sp = predictor(image, rect)
  landmarks = np.array([[p.x, p.y] for p in sp.parts()])
  x = []
  y_alis = []
  y = []
  w = []
  h = []

  x.append(landmarks[1][0])
  y_alis.append(landmarks[17][1])
  y_alis.append(landmarks[18][1])
  y_alis.append(landmarks[23][1])
  y_alis.append(landmarks[24][1])
  w.append(landmarks[15][0])
  h.append(landmarks[8][1])
  y.append(min(y_alis))

  crop_img = image[y[0]:h[0], x[0]:w[0]]
  
  return crop_img


def preprocessing (path,sig,size_x,size_y):
  #face_pre = []
  #face = []
  global face_pre,crop_img
  base_image = cv2.imread(path)
  if(base_image is not None):
    grey = cv2.cvtColor(base_image, cv2.COLOR_BGR2GRAY)
    #for (x, y, w, h) in faces:
      #crop_img = grey[y:y + h, x:x + w]
    #crop_img = grey[faces[0,1]:faces[0,1]+faces[0,3], faces[0,0]:faces[0,0]+faces[0,3]]
    crop_img = cropping(grey)
    face_pre = cv2.GaussianBlur(crop_img, ksize=(0, 0), sigmaX=sig, borderType=cv2.BORDER_REPLICATE)
    face = np.double(cv2.resize(np.array(face_pre),(size_x,size_y)))
  return face

def feature(flash,background):
  a = flash - background
  b = flash + background
  c = a/b
  trans = np.transpose(c)
  feat_vec = np.reshape(trans, (1, trans.size))
  feat_vec = np.nan_to_num(feat_vec)
  return feat_vec

def diffuse_extract(output,path_flash,path_bg):
  output_folder = output
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  for i in range(len(path_flash)): #range(20):  #range(len(path_flash)):
    flash = preprocessing(path_flash[i],5,100,100)
    background = preprocessing(path_bg[i],5,100,100)
    pantulan = feature(flash,background)
    name = os.path.join(base_path,output_folder,str(i)+'.JPG')
    print(name)
    plt.imsave(name, pantulan, cmap='gray')

def save_images(path_flash,path_bg,file_name):
  feature_label = []
  for i in range(len(path_flash)):
    flash = preprocessing(path_flash[i],5,100,100)
    background = preprocessing(path_bg[i],5,100,100)
    pantulan = feature(flash,background)
    print(path_flash[i])
    label = 0
    feature_label.append(np.append(pantulan,np.array(label)))
    np.save(file_name,np.array(feature_label))

## Build .npy dataset file

In [ ]:
save_images(live_flash,live_bg,"live.npy")

In [ ]:
save_images(spoof_flash,spoof_bg,"spoof.npy")